### ESCOM - IIA
#### FUNDAMENTOS DE INTELIGENCIA ARTIFICIAL - PROYECTO ML
#### Semestre 2023-2 - Grupo 4BV2
--- 
##### Autor:
- **Valdés Luis Eliot Fabián**

In [1]:
# importamos la librerias necesarias
from ucimlrepo import fetch_ucirepo  # libreria de repositorios de datasets
import pandas as pd  # libreria para el manejo de dataframes

----
# PASO 1: Carga de datos
Sección inicial para obtener el dataset, describirlo de manera general usando estadisticas

In [2]:
# definimos los datasets a utilizar
datasets = {
    "Eliot": 53, # Iris
    "Ethel": 17, # Breast Cancer
    "Leo": 878, # Cirrhosis
    "Adair": 109, # Wine
}
dataset = fetch_ucirepo(id=datasets["Eliot"]) # cargamos el dataset que deseemos utilizar

In [3]:
# mostramos un pequeño resumen de lo que trata el dataset 
dataset.metadata.abstract

'A small classic dataset from Fisher, 1936. One of the earliest known datasets used for evaluating classification methods.\n'

In [4]:
# obtenemos los datos
X = dataset.data.features 
y = dataset.data.targets 

In [5]:
# creamos un dataframe con los datos
df = pd.DataFrame(X, columns=dataset.data.feature_names)
# agregamos la columna target
df['target'] = y

# mostramos la cantidad de registros
print(f'Cantidad de registros: {len(df)}')

# imprimimos los primeros 5 registros
df.head()

Cantidad de registros: 150


,sepal length,sepal width,petal length,petal width,target
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [6]:
# mostramos los tipos de datos de cada columna
print(f"Tipos de datos (Metodo pandas):\n{df.dtypes}\n")

Tipos de datos (Metodo pandas):
sepal length    float64
sepal width     float64
petal length    float64
petal width     float64
target           object
dtype: object



In [7]:
# mostramos los tipos de datos usando los metodos de la librearia de repositorios
dataframe_tipos = pd.DataFrame({'Variable Name': dataset.variables['name'], 'Type': dataset.variables['type']})
print(f"Tipos de datos (Metodo ucimlrepo):\n{dataframe_tipos}\n")

Tipos de datos (Metodo ucimlrepo):
  Variable Name         Type
0  sepal length   Continuous
1   sepal width   Continuous
2  petal length   Continuous
3   petal width   Continuous
4         class  Categorical



In [8]:
# por cada columna hacer lo siguiente 
# si el tipo de dato es numerico, mostrar la media, mediana, desviacion estandar, minimo y maximo
# si el tipo de dato es categorico, mostrar la cantidad de valores unicos

# limitamos la cantidad de columnas a 10
for col in df.columns[:10]:
    if df[col].dtype == 'float64' or df[col].dtype == 'int64':
        print(f"=> '{col}':")
        print(f"\t-Media: {df[col].mean()}")
        print(f"\t-Mediana: {df[col].median()}")
        print(f"\t-Desviacion estandar: {df[col].std()}")
        print(f"\t-Minimo: {df[col].min()}")
        print(f"\t-Maximo: {df[col].max()}")
    else:
        print(f"=> '{col}':")
        print(f"\t-Valores unicos: {df[col].unique()}")
        print(f"\t-Cantidad de valores unicos: {df[col].nunique()}")

=> 'sepal length':
	-Media: 5.843333333333334
	-Mediana: 5.8
	-Desviacion estandar: 0.828066127977863
	-Minimo: 4.3
	-Maximo: 7.9
=> 'sepal width':
	-Media: 3.0540000000000003
	-Mediana: 3.0
	-Desviacion estandar: 0.4335943113621737
	-Minimo: 2.0
	-Maximo: 4.4
=> 'petal length':
	-Media: 3.758666666666666
	-Mediana: 4.35
	-Desviacion estandar: 1.7644204199522626
	-Minimo: 1.0
	-Maximo: 6.9
=> 'petal width':
	-Media: 1.1986666666666668
	-Mediana: 1.3
	-Desviacion estandar: 0.7631607417008411
	-Minimo: 0.1
	-Maximo: 2.5
=> 'target':
	-Valores unicos: ['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']
	-Cantidad de valores unicos: 3


----

# PASO 2: PREPROCESAMIENTO DE DATOS

Sección para preprocesar los datos, aquí separamos el dataset en los vectores de entrada X & de salida Y(las clases que son definidas con tipo de dato categorico)


In [9]:
# funcion para separar cualquier dataframe en dos vectores (entrada y salida)
def separate_dataframe(df):
    vector_x = df.drop(df.columns[-1], axis=1) # obtencion de variables de entrada
    vector_y = df[df.columns[-1]] # obtencion de variable de salida (clase/target)
    return vector_x, vector_y

# separamos el dataframe en dos vectores, uno con las variables independientes y otro con la variable dependiente
vector_x, vector_y = separate_dataframe(df)

In [10]:
# mostramos los primeros 5 registros del vector de entrada X
vector_x.head()

,sepal length,sepal width,petal length,petal width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [11]:
# mostramos los primeros 5 registros del vector de salida Y
vector_y.head()

0    Iris-setosa
1    Iris-setosa
2    Iris-setosa
3    Iris-setosa
4    Iris-setosa
Name: target, dtype: object

In [12]:
# mostramos los valores unicos del vector de salida Y
vector_y.unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [13]:
# creamos una funcion que recibe como parametros el vector de entrada X y el vector de salida Y
def describe_categories(vector_x, vector_y):
    # agrupamos el vector de entrada X de acuerdo al vector de salida Y
    grouped = vector_x.groupby(vector_y)
    for name, group in grouped:
        print("\n",("="*50))
        print(f'Clase: {name}')
        print("="*50)
        # por cada grupo, accedemos a cada una de las columnas
        for col in group.columns:
            # validamos que el tipo de dato de la columna sea numerico y de ser el caso mosstramos las estadisticas
            if group[col].dtype == 'float64' or group[col].dtype == 'int64':
                print(f"=> '{col}':")
                print(f"\t-Media: {group[col].mean()}")
                print(f"\t-Mediana: {group[col].median()}")
                print(f"\t-Desviacion estandar: {group[col].std()}")
                print(f"\t-Minimo: {group[col].min()}")
                print(f"\t-Maximo: {group[col].max()}")
            else:
                print(f"=> '{col}':")
                print(f"\tValores unicos: {group[col].unique()}")
                print(f"\tCantidad de valores unicos: {group[col].nunique()}")            
        

describe_categories(vector_x, vector_y)


Clase: Iris-setosa
=> 'sepal length':
	-Media: 5.006
	-Mediana: 5.0
	-Desviacion estandar: 0.35248968721345136
	-Minimo: 4.3
	-Maximo: 5.8
=> 'sepal width':
	-Media: 3.418
	-Mediana: 3.4
	-Desviacion estandar: 0.38102439795469095
	-Minimo: 2.3
	-Maximo: 4.4
=> 'petal length':
	-Media: 1.464
	-Mediana: 1.5
	-Desviacion estandar: 0.17351115943644546
	-Minimo: 1.0
	-Maximo: 1.9
=> 'petal width':
	-Media: 0.244
	-Mediana: 0.2
	-Desviacion estandar: 0.1072095030816784
	-Minimo: 0.1
	-Maximo: 0.6

Clase: Iris-versicolor
=> 'sepal length':
	-Media: 5.936
	-Mediana: 5.9
	-Desviacion estandar: 0.5161711470638634
	-Minimo: 4.9
	-Maximo: 7.0
=> 'sepal width':
	-Media: 2.7700000000000005
	-Mediana: 2.8
	-Desviacion estandar: 0.3137983233784114
	-Minimo: 2.0
	-Maximo: 3.4
=> 'petal length':
	-Media: 4.26
	-Mediana: 4.35
	-Desviacion estandar: 0.46991097723995795
	-Minimo: 3.0
	-Maximo: 5.1
=> 'petal width':
	-Media: 1.3259999999999998
	-Mediana: 1.3
	-Desviacion estandar: 0.19775268000454405
	-Min

-----
# PASO 3: LIMPIEZA DE DATASET
Sección para limpiar el dataset para eliminar valores que no aportan al modelo, como los valores nulos o NaN.

In [14]:
# juntamos los vectores de entrada y salida en un solo dataframe
df = vector_x.join(vector_y)
# mostramos la cantidad de registros
print(f'Cantidad de registros (Before cleaning): {len(df)}')

Cantidad de registros (Before cleaning): 150


In [15]:
# eliminamos los registros que tengan valores nulos en alguna de las columnas
df = df.dropna()
# mostramos la cantidad de registros
print(f'Cantidad de registros: (After cleaning) {len(df)}')

Cantidad de registros: (After cleaning) 150


In [16]:
# de nuevo separamos el dataframe en dos vectores, uno con las variables independientes y otro con la variable dependiente
vector_x, vector_y = separate_dataframe(df)

In [17]:
# conservamos unicamente las columnas con tipo de dato Inter o Continuous
vector_x = vector_x.select_dtypes(include=['int64', 'float64'])
# mostramos las primeras 5 filas del vector de entrada
vector_x.head()

,sepal length,sepal width,petal length,petal width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [18]:
print(f'Otra forma de limpiar el vector de entrada. Usa el dataset original y los tipos de datos del dataset pero no es tan eficiente como el metodo anterior ya que alginas columnas con tipo de dato numerico no son numericas en el dataset')
"""
for col_name in vector_x.columns:
    # verificar el tipo de dato de cada columna usando el dataset
    if col_name in dataset.variables['name'].values:
        # obtener el tipo de dato de la columna en el dataset
        col_type = dataset.variables['type'][dataset.variables['name'] == col_name].values[0]
        # eliminamos del vector_x aquellas columnas que no son numericas
        if col_type != 'Integer' and col_type != 'Continuous':
            vector_x = vector_x.drop(col_name, axis=1)
        else:        
            # parseamos las columnas a tipo de dato float64
            vector_x[col_name] = vector_x[col_name].astype('float64')                    

# mostramos las primeras 5 filas del vector de entrada
vector_x.head()                    
"""

Otra forma de limpiar el vector de entrada. Usa el dataset original y los tipos de datos del dataset pero no es tan eficiente como el metodo anterior ya que alginas columnas con tipo de dato numerico no son numericas en el dataset


"\nfor col_name in vector_x.columns:\n    # verificar el tipo de dato de cada columna usando el dataset\n    if col_name in dataset.variables['name'].values:\n        # obtener el tipo de dato de la columna en el dataset\n        col_type = dataset.variables['type'][dataset.variables['name'] == col_name].values[0]\n        # eliminamos del vector_x aquellas columnas que no son numericas\n        if col_type != 'Integer' and col_type != 'Continuous':\n            vector_x = vector_x.drop(col_name, axis=1)\n        else:        \n            # parseamos las columnas a tipo de dato float64\n            vector_x[col_name] = vector_x[col_name].astype('float64')                    \n\n# mostramos las primeras 5 filas del vector de entrada\nvector_x.head()                    \n"

-----
# PASO 4: IMPLEMENTACIÓN DE MODELOS DE ML Y EVALUACIÓN CON DIFERENTES MÉTRICAS
Sección para implementar los modelos Minima Distancia y KNN (K=1) para evaluar con los metodos: train-test split, k-fold cross validation y bootstrapping.

### MODELOS

#### - DESARROLLO DE MODELO MINIMA DISTANCIA

In [19]:
def calcular_centroides(X, y):
    clases = set(y)
    centroides = {c: [0] * len(X[0]) for c in clases}
    contador = {c: 0 for c in clases}

    for xi, yi in zip(X, y):
        contador[yi] += 1
        for i in range(len(xi)):
            centroides[yi][i] += xi[i]

    for c in centroides:
        centroides[c] = [x / contador[c] for x in centroides[c]]
    
    return centroides

def clasificador_minima_distancia(X, centroides):
    predicciones = []
    for xi in X:
        distancias = {c: sum((xi[j] - centroides[c][j])**2 for j in range(len(xi))) for c in centroides}
        predicciones.append(min(distancias, key=distancias.get))
    return predicciones


#### DESARROLLO DE MODELO K NEAREST NEIGHBORS (K=1)

In [20]:
def clasificador_knn(X_train, y_train, X_test):
    predicciones = []
    for xi in X_test:
        distancias = [sum((xi[j] - X_train[i][j])**2 for j in range(len(xi))) for i in range(len(X_train))]
        min_index = distancias.index(min(distancias))
        predicciones.append(y_train[min_index])
    return predicciones

### MÉTODOS DE VALIDACIÓN

#### - DESARROLLO DE MÉTODO TEST-TRAIN SPLIT (80-20)

In [21]:
def entrenamiento_prueba(X, y, porcentaje_prueba):
    indice_corte = int(len(X) * (1 - porcentaje_prueba))
    X_train = X[:indice_corte]
    y_train = y[:indice_corte]
    X_test = X[indice_corte:]
    y_test = y[indice_corte:]
    return X_train, y_train, X_test, y_test

#### - DESARROLLO DE MÉTODO K-FOLD CROSS VALIDATION (K=5)

In [22]:
def k_fold_cross_validation(X, y, k):
    tamaño_fold = len(X) // k
    for i in range(k):
        inicio = i * tamaño_fold
        fin = (i + 1) * tamaño_fold if i != k - 1 else len(X)
        X_train = X[:inicio] + X[fin:]
        y_train = y[:inicio] + y[fin:]
        X_test = X[inicio:fin]
        y_test = y[inicio:fin]
        yield X_train, y_train, X_test, y_test

#### - DESARROLLO DE MÉTODO BOOTSTRAPPING (B=100)

In [23]:
import random

def bootstrap(X, y, n):
    for _ in range(n):
        indices = [random.randint(0, len(X) - 1) for _ in range(len(X))]
        X_train = [X[i] for i in indices]
        y_train = [y[i] for i in indices]
        X_test = [X[i] for i in range(len(X)) if i not in indices]
        y_test = [y[i] for i in range(len(X)) if i not in indices]
        yield X_train, y_train, X_test, y_test

### FUNCIONES AUXILIARES

#### - FUNCION PARA CALCULAR LA RELACIÓN EFIENCIA-ERROR DE UN MODELO

In [24]:
def calcular_eficiencia_error(y_real, y_pred):
    correctos = sum(1 for real, pred in zip(y_real, y_pred) if real == pred)
    eficiencia = correctos / len(y_real)
    error = 1 - eficiencia
    return eficiencia, error

#### - FUNCION PARA TRANSFORMAR UN DATAFRAME EN UNA LISTA DE LISTAS

In [25]:
def transformar_datos(vector_x, vector_y):
    X = vector_x.values.tolist() 
    # Transformar vector_y a una lista
    y = vector_y.values.tolist() 
    y = [item[0] for item in y] if isinstance(y[0], list) else y
    return X, y

- ### 4 => CONTENEDOR DE EJECUCIÓN DE MODELO: DISTANCIA MINIMA

In [26]:
def run_min_distance_model(X, y):
    # ENTRENAMIENTO Y PRUEBA
    X_train, y_train, X_test, y_test = entrenamiento_prueba(X, y, porcentaje_prueba=0.2)
    # Calcular centroides y hacer predicciones
    centroides = calcular_centroides(X_train, y_train)
    predicciones = clasificador_minima_distancia(X_test, centroides)
    # Calcular eficiencia y error
    eficiencia, error = calcular_eficiencia_error(y_test, predicciones)
    print(f"Minima distancia: eficiencia = {eficiencia}, error = {error} - Metodo de entrenamiento y prueba")
    
    # K-FOLD CROSS VALIDATION
    eficiencias = []
    errores = []
    k = 5  # Número de folds
    for X_train, y_train, X_test, y_test in k_fold_cross_validation(X, y, k):
        centroides = calcular_centroides(X_train, y_train)
        predicciones = clasificador_minima_distancia(X_test, centroides)
        eficiencia, error = calcular_eficiencia_error(y_test, predicciones)
        eficiencias.append(eficiencia)
        errores.append(error)
    # Calcular promedios
    eficiencia_promedio = sum(eficiencias) / k
    error_promedio = sum(errores) / k
    print(f"Minima distancia: eficiencia = {eficiencia_promedio}, error = {error_promedio} - K-fold cross validation")
    
    # BOOTSTRAP
    eficiencias = []
    errores = []
    n_iteraciones = 100  # Número de iteraciones de Bootstrap
    for X_train, y_train, X_test, y_test in bootstrap(X, y, n_iteraciones):
        centroides = calcular_centroides(X_train, y_train)
        predicciones = clasificador_minima_distancia(X_test, centroides)
        eficiencia, error = calcular_eficiencia_error(y_test, predicciones)
        eficiencias.append(eficiencia)
        errores.append(error)
    # Calcular promedios
    eficiencia_promedio = sum(eficiencias) / n_iteraciones
    error_promedio = sum(errores) / n_iteraciones
    print(f"Minima distancia: eficiencia = {eficiencia_promedio}, error = {error_promedio} - Bootstrap")      


- ### 5 => CONTENDOR DE EJECUCIÓN DE MODELO: KNN(K=1)

In [27]:
def run_knn_model(X, y):
    # ENTRENAMIENTO Y PRUEBA
    # Dividir los datos
    X_train, y_train, X_test, y_test = entrenamiento_prueba(X, y, porcentaje_prueba=0.2)
    # Hacer predicciones
    predicciones = clasificador_knn(X_train, y_train, X_test)
    # Calcular eficiencia y error
    eficiencia, error = calcular_eficiencia_error(y_test, predicciones)
    print(f"KNN: eficiencia = {eficiencia}, error = {error} - Metodo de entrenamiento y prueba")
    
    # K-FOLD CROSS VALIDATION
    eficiencias = []
    errores = []
    k = 5  # Número de folds
    for X_train, y_train, X_test, y_test in k_fold_cross_validation(X, y, k):
        predicciones = clasificador_knn(X_train, y_train, X_test)
        eficiencia, error = calcular_eficiencia_error(y_test, predicciones)
        eficiencias.append(eficiencia)
        errores.append(error)
    # Calcular promedios
    eficiencia_promedio = sum(eficiencias) / k
    error_promedio = sum(errores) / k
    print(f"KNN: eficiencia = {eficiencia_promedio}, error = {error_promedio} - K-fold cross validation")
    
    # BOOTSTRAP
    eficiencias = []
    errores = []
    n_iteraciones = 10  # Número de iteraciones de Bootstrap
    for X_train, y_train, X_test, y_test in bootstrap(X, y, n_iteraciones):
        predicciones = clasificador_knn(X_train, y_train, X_test)
        eficiencia, error = calcular_eficiencia_error(y_test, predicciones)
        eficiencias.append(eficiencia)
        errores.append(error)
    # Calcular promedios
    eficiencia_promedio = sum(eficiencias) / n_iteraciones
    error_promedio = sum(errores) / n_iteraciones
    print(f"KNN: eficiencia = {eficiencia_promedio}, error = {error_promedio} - Bootstrap")       

### EJECUCIÓN DE MODELOS

In [28]:
# transformamos los vectores de entrada y salida a listas
X, y = transformar_datos(vector_x, vector_y)

-----
# PASO 4: EJECUCIÓN DE DISTANCIA MINIMA
- #### 4.A => DISTANCIA MINIMA - EVALUADO CON: TRAIN-TEST SPLIT
- #### 4.B => DISTANCIA MINIMA - EVALUADO CON: K FOLD CROSS VALIDATION
- #### 4.C => DISTANCIA MINIMA - EVALUADO CON: BOOTSTRAP

In [29]:
# ejecutamos el modelo de distancia minima y validamos internamente con cada metodo
run_min_distance_model(X, y)

Minima distancia: eficiencia = 0.8666666666666667, error = 0.1333333333333333 - Metodo de entrenamiento y prueba
Minima distancia: eficiencia = 0.9133333333333333, error = 0.08666666666666664 - K-fold cross validation
Minima distancia: eficiencia = 0.9223099260783647, error = 0.07769007392163518 - Bootstrap


-----
# PASO 5: EJECUCIÓN DE KNN(K=1)
- #### 5.A => KNN - EVALUADO CON: TRAIN-TEST SPLIT
- #### 5.B => KNN - EVALUADO CON: K FOLD CROSS VALIDATION
- #### 5.C => KNN - EVALUADO CON: BOOTSTRAP

In [30]:
# ejecutamos el modelo de KNN y validamos internamente con cada metodo
run_knn_model(X, y)

KNN: eficiencia = 0.8333333333333334, error = 0.16666666666666663 - Metodo de entrenamiento y prueba
KNN: eficiencia = 0.9266666666666665, error = 0.07333333333333332 - K-fold cross validation
KNN: eficiencia = 0.9526223351735024, error = 0.04737766482649758 - Bootstrap


-----
# PASO 6: ELIMINACIÓN DE ATRIBUTOS

In [31]:
# seleccionamos el atributo 1 (columna) de mi vector_x de entradas
atributo_1 = vector_x.columns[1]
# seleccionamos el atributo 2 (columna) de mi vector_x de entradas
atributo_2 = vector_x.columns[2]

In [32]:
# eliminamos el atributo_1 pero de un vector de entradas de prueba
vector_x_prueba_1 = vector_x.drop(atributo_1, axis=1)

# repetimos proceso para hacer test de modelo
X, y = transformar_datos(vector_x_prueba_1, vector_y)
run_min_distance_model(X, y)
run_knn_model(X, y)

Minima distancia: eficiencia = 0.8666666666666667, error = 0.1333333333333333 - Metodo de entrenamiento y prueba
Minima distancia: eficiencia = 0.9199999999999999, error = 0.07999999999999999 - K-fold cross validation
Minima distancia: eficiencia = 0.9234420038152102, error = 0.0765579961847898 - Bootstrap
KNN: eficiencia = 0.8333333333333334, error = 0.16666666666666663 - Metodo de entrenamiento y prueba
KNN: eficiencia = 0.9333333333333332, error = 0.06666666666666665 - K-fold cross validation
KNN: eficiencia = 0.9516747565586575, error = 0.048325243441342515 - Bootstrap


In [33]:
# eliminamos el atributo_2 de mi vector de entradas pero de un vector de entradas de prueba 2
vector_x_prueba_2 = vector_x.drop(atributo_2, axis=1)

# repetimos proceso para hacer test de modelo
X, y = transformar_datos(vector_x_prueba_2, vector_y)
run_min_distance_model(X, y)
run_knn_model(X, y)

Minima distancia: eficiencia = 0.8, error = 0.19999999999999996 - Metodo de entrenamiento y prueba
Minima distancia: eficiencia = 0.8666666666666668, error = 0.1333333333333333 - K-fold cross validation
Minima distancia: eficiencia = 0.8674471681316213, error = 0.13255283186837896 - Bootstrap
KNN: eficiencia = 0.8666666666666667, error = 0.1333333333333333 - Metodo de entrenamiento y prueba
KNN: eficiencia = 0.9066666666666666, error = 0.09333333333333331 - K-fold cross validation
KNN: eficiencia = 0.9482128153189973, error = 0.051787184681002865 - Bootstrap


In [34]:
# eliminamos los atributos 1 y 2 de mi vector de entradas pero de un vector de entradas de prueba 3
vector_x_prueba_3 = vector_x.drop([atributo_1, atributo_2], axis=1)

# repetimos proceso para hacer test de modelo
X, y = transformar_datos(vector_x_prueba_3, vector_y)
run_min_distance_model(X, y)
run_knn_model(X, y)

Minima distancia: eficiencia = 0.7666666666666667, error = 0.23333333333333328 - Metodo de entrenamiento y prueba
Minima distancia: eficiencia = 0.8466666666666667, error = 0.1533333333333333 - K-fold cross validation
Minima distancia: eficiencia = 0.8488643631511624, error = 0.15113563684883755 - Bootstrap
KNN: eficiencia = 0.8333333333333334, error = 0.16666666666666663 - Metodo de entrenamiento y prueba
KNN: eficiencia = 0.8933333333333333, error = 0.10666666666666666 - K-fold cross validation
KNN: eficiencia = 0.9285197320283866, error = 0.07148026797161355 - Bootstrap
